Cruise through data :D

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
#Read data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
print(train.shape, test.shape)

In [ ]:
pd.set_option('display.max_rows', 500)
train.sample(10)

In [ ]:
cat_data = train.select_dtypes(include=['object'])
cat_cols = cat_data.columns
print(cat_cols)
cat_data.head()
#Interesting labels

In [ ]:
#Lets get binary variables
train.apply(lambda x: x.unique().shape[0])
#All columns from X10 -X385 are binary
#Basically, all variables are cardinal

In [ ]:
#Lets encode categoricals to numerical labels
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
lb = LabelEncoder()
lb_dict = defaultdict(LabelEncoder)
all_data = pd.concat([train, test])

for col in cat_cols:
    lb_dict[col] = lb.fit(all_data[col])
    train[col + '_mod'] = lb_dict[col].transform(train[col])
    test[col + '_mod'] = lb_dict[col].transform(test[col])
    


In [ ]:

#Set train and test arrays
feats = [f for f in train.columns if f not in list(cat_cols) + ['y','ID']]

X_train, X_test = train[feats], test[feats]
y_train = train['y']

In [ ]:
#Extra tree regressor 

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, stratify=X_train, test_size=0.2, random_state=1)

etr = ExtraTreesRegressor(n_estimators=500, n_jobs=-1, max_depth=4, random_state=1)
etr.fit(X_tr, y_tr)
print(etr.score(X_val, y_val))

In [ ]:
importances = etr.feature_importances_
std = np.std([tree.feature_importances_ for tree in etr.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1][:50]

plt.figure(figsize=(18,16))
plt.bar(range(len(indices)), importances[indices], color="black", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), np.array(feats)[indices], rotation=90)
plt.show() #Paint it black

**X314, X315, seem to be best features** - Will need to do some work to get value out of other binary ones

In [ ]:
sns.boxplot(x='X314', y='y', data=train)
plt.show()

In [ ]:
sns.boxplot(x='X315', y='y', data=train)
plt.show()

In [ ]:
sns.boxplot(x='X47', y='y', data=train)
plt.show()

In [ ]:
plt.hist(train.y, bins=100)
plt.show()

In [ ]:
from sklearn import metrics
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'error', metrics.r2_score(labels, preds)
model_xgb = XGBRegressor(n_estimators=1500, learning_rate=0.02, max_depth=4)

model_xgb.fit(X_tr, y_tr, verbose=10, eval_set=[(X_val, y_val)], eval_metric=evalerror)

In [ ]:
from sklearn.model_selection import StratifiedKFold:
 